# XSS Attack Detection - Model Testing

# 📚 Import Libraries

In [1]:
import numpy as np
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-04-28 16:55:17.266350: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 16:55:17.272401: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 16:55:17.283945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745855717.309746  111253 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745855717.315099  111253 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745855717.331996  111253 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# 1️⃣ Load the Saved Model and Tokenizer

In [2]:
# Paths to the model and tokenizer
MODEL_PATH = 'xss_detector_model.h5'
TOKENIZER_PATH = 'tokenizer.pickle'
MAX_LENGTH = 200  # Must match training

# Load the trained model
model = load_model(MODEL_PATH)
print("✅ Model loaded successfully!")

# Load the saved tokenizer
with open(TOKENIZER_PATH, 'rb') as handle:
    tokenizer = pickle.load(handle)
print("✅ Tokenizer loaded successfully!")

2025-04-28 16:55:26.113545: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


✅ Model loaded successfully!
✅ Tokenizer loaded successfully!


# 2️⃣ Define the Prediction Function

In [24]:
def predict_xss(sentence):
    """
    Predicts if the given sentence contains an XSS attack.
    """
    # Preprocess input
    sequence = tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')
    
    # Make prediction
    prediction = model.predict(padded)[0][0]
    
    # Display results
    print(f"\nInput Sentence: {sentence}")
    print(f"XSS Probability: {prediction:.4f}")
    
    if prediction > 0.5:
        print("⚠️  Detected as XSS Attack")
    else:
        print("✅  Safe Sentence")

# 3️⃣ Test the Model on New Inputs

In [25]:
# Example 1: Safe input
safe_sentence = "Welcome to our website! We hope you enjoy your stay."
predict_xss(safe_sentence)

# Example 2: XSS payload
xss_payload = "<img src=x onerror=alert(1)>"
predict_xss(xss_payload)

# Example 3: Another XSS attempt
another_xss = "<script>alert('XSS');</script>"
predict_xss(another_xss)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step

Input Sentence: Welcome to our website! We hope you enjoy your stay.
XSS Probability: 0.0001
✅  Safe Sentence
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step

Input Sentence: <img src=x onerror=alert(1)>
XSS Probability: 1.0000
⚠️  Detected as XSS Attack
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step

Input Sentence: <script>alert('XSS');</script>
XSS Probability: 1.0000
⚠️  Detected as XSS Attack
